In [5]:
import pandas as pd
import plotly.express as px
from src import config
from tqdm import tqdm
from src.utils.llm.openai_client import get_proposer
predictions = pd.read_parquet("output/predictions.parquet")

In [11]:
from src.feature_engineering.categories import get_category_column
from src.feature_engineering.legislature_period import get_legislature_period_metadata
from src.feature_engineering.mirror_beschlussempfehlung import prepare_final_dataset


dataset = prepare_final_dataset()
dataset["category"] = get_category_column(dataset["summary_embedding"])

dataset["metadata"] = dataset.apply(
    lambda row: get_legislature_period_metadata(row["party"], row["date"]), axis=1
)
dataset[[
    "is_governing",
    "bundestag",
]] = dataset["metadata"].apply(pd.Series)
dataset.drop(columns=["metadata"], inplace=True)
dataset["vote_correct"] = (dataset["prediction"] == dataset["ground_truth"])

2025-05-31 12:21:00.906 | INFO     | src.feature_engineering.categories:get_category_column:36 - Calculating closest categories through embeddings
2025-05-31 12:21:00.907 | INFO     | src.feature_engineering.categories:get_embeddings:13 - Loading existing categories embeddings...


In [45]:
from src.votes.build import is_own_proposal


votes = pd.read_parquet("output/votes.parquet")

votes


,vote_id,type,drucksache_title,drucksache_id,beschlussempfehlung,content,summary,summary_embedding,date,proposers
0,20250318_3,Gesetzentwurf,Gesetzentwurf der Fraktionen der SPD und CDU/C...,20/15096,None,A. Problem Fundamentale Veränderungen der Sich...,Abgestimmt wird über die Einrichtung einer ver...,"[0.009540573693811893, 0.03497336432337761, 0....",2025-03-18,"[SPD, Union]"
1,20250318_2,Änderungsantrag,Änderungsantrag der Fraktion der FDP zu der zw...,20/15120,None,Deutscher Bundestag Drucksache 20/15120 20. Wa...,"Der Bundestag soll beschließen, das Grundgeset...","[0.011175020597875118, 0.03776725009083748, 0....",2025-03-18,[FDP]
2,20250318_1,Gesetzentwurf,Gesetzentwurf der Fraktion der FDP Entwurf ein...,20/15099,None,A. Problem Der russische Angriffskrieg gegen d...,Im Bundestag wird über die Erweiterung des Son...,"[0.012960884720087051, 0.037686433643102646, 0...",2025-03-18,[FDP]
3,20250131_1,Gesetzentwurf,Gesetzentwurf der Fraktion der CDU/CSU Entwurf...,20/12804,None,A. Problem § 1 Absatz 1 Satz 1 bis 3 des Aufen...,Das Gesetz soll das Ziel der Begrenzung der Zu...,"[0.04290461167693138, 0.07238075137138367, 0.0...",2025-01-31,[Union]
4,20241219_1,Gesetzentwurf,"Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜN...",20/12977,None,A. Problem und Ziel Das Grundgesetz hat sich f...,Im Bundestag wird über einen Gesetzentwurf abg...,"[-0.013081829994916916, 0.0215305108577013, 0....",2024-12-19,"[SPD, Union, DIE_GRÜNEN, FDP]"
...,...,...,...,...,...,...,...,...,...,...
658,20130131,Antrag,Antrag der Bundesregierung Fortsetzung der Bet...,17/11685,Annahme,Deutscher Bundestag Drucksache 17/11685 17. Wa...,Der Bundestag stimmt der Fortsetzung des Einsa...,"[-0.018130291253328323, 0.036971572786569595, ...",2013-01-31,[Bundesregierung]
659,20121214_1,Antrag,Antrag der Bundesregierung Entsendung bewaffne...,17/11783,Annahme,Deutscher Bundestag Drucksache 17/11783 17. Wa...,Der Bundestag stimmt der Entsendung bewaffnete...,"[-0.03251594305038452, 0.022284308448433876, 0...",2012-12-14,[Bundesregierung]
660,20121213_4,Antrag,Antrag der Bundesregierung Fortsetzung des Ein...,17/11466,Annahme,Deutscher Bundestag Drucksache 17/11466 17. Wa...,Der Bundestag stimmt der Fortsetzung des Einsa...,"[-0.00255092466250062, 0.07303918898105621, 0....",2012-12-13,[Bundesregierung]
661,20121213_2,Antrag,"Antrag der Abgeordneten Johanna Voß, Ulla Lötz...",17/11328,Ablehnung,Deutscher Bundestag Drucksache 17/11328 17. Wa...,Abgestimmt wird über ein Gesetz zur vollständi...,"[-0.004911317024379969, 0.06889131665229797, 0...",2012-12-13,[DIE_LINKE]


In [1]:
from src.run_preprocessing import run_preprocessing

run_preprocessing()

2025-05-31 14:20:29.937 | INFO     | src.votes.gather:scrape_urls:56 - Vote metadata already exists. Skipping scraping.
2025-05-31 14:20:30.006 | INFO     | src.manifestos.download:download_manifestos:40 - Manifesto for FDP 2013 already downloaded. Skipping.
2025-05-31 14:20:30.007 | INFO     | src.manifestos.download:download_manifestos:40 - Manifesto for FDP 2017 already downloaded. Skipping.
2025-05-31 14:20:30.008 | INFO     | src.manifestos.download:download_manifestos:40 - Manifesto for FDP 2021 already downloaded. Skipping.
2025-05-31 14:20:30.009 | INFO     | src.manifestos.download:download_manifestos:40 - Manifesto for FDP 2025 already downloaded. Skipping.
2025-05-31 14:20:30.010 | INFO     | src.manifestos.download:download_manifestos:40 - Manifesto for SPD 2009 already downloaded. Skipping.
2025-05-31 14:20:30.010 | INFO     | src.manifestos.download:download_manifestos:40 - Manifesto for SPD 2013 already downloaded. Skipping.
2025-05-31 14:20:30.011 | INFO     | src.manif

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Building vote entrypoints:  69%|██████▉   | 489/706 [08:24<02:20,  1.54vote/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Building vote entrypoints:  74%|███████▍  | 522/706 [09:01<04:40,  1.53s/vote]2025-05-31 14:29:31.692 | ERROR    | src.votes.build:build_entrypoints:103 - Error building vote 522: Title not found in 522
2025-05-31 14:29:31.746 | ERROR    | src.votes.build:build_entrypoints:103 - Error building vote 523: Title not found in 523
Building vote entrypoints:  86%|████████▌ | 605/706 [10:28<02:04,  1.23s/vote]

...

Building vote entrypoints:  91%|█████████ | 642/706 [11:03<01:02,  1.02vote/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Building vote entrypoints:  93%|█████████▎| 654/706 [11:18<00:59,  1.14s/vote]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Building vote entrypoints: 100%|██████████| 706/706 [12:01<00:00,  1.02s/vote]
2025-05-31 14:32:32.573 | WARNING  | src.drucksachen.beschlussempfehlung:parse:56 - No votes found in Beschlussempfehlung 20/10055
2025-05-31 14:32:32.574 | WARNING  | src.drucksachen.beschlussempfehlung:build:84 - No underlying votes found for 41 with drucksache 20/10055.
2025-05-31 14:32:32.653 | WARNING  | src.drucksachen.beschlussempfehlung:parse:56 - No votes found in Beschlussempfehlung 20/10410
2025-05-31 14:32:32.654 | WARNING  | src.drucksachen.beschlussempfehlung:build:84 - No underlying votes found for 45 with drucksache 20/10410.
2025-05-31 14:32:32.693 | WARNING  | src.drucksachen.beschlussempfehlung:parse:56 - No votes found in Beschlussempfehlung 20/9676
2025-05-31 14:32:32.694 | WARNING  | src.drucksachen.beschlussempfehlung:build:84 - No underlying votes found for 61 with drucksache 20/9676.
2025-05-31 14:32:32.978 | WARNING  | src.drucksachen.beschlussempfehlung:parse:56 - No votes found in